In [ ]:
nome ='Gabriel Victor de Sousa Lopes'
print(f'Meu nome é {nome}')

Meu nome é Gabriel Victor de Sousa Lopes


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 55.6 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Mon Jun  6 21:24:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
# from typing import List


# def tokenize(text: str, tokenizer):
#     # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
#     return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


# def truncate_and_pad(batch_word_ids, pad_token_id, seq_length=9):  

#     # TODO: Truncate.
#     lens = [len(l) for l in batch_word_ids] 
#     arr_zeros = np.zeros((len(batch_word_ids),seq_length),int) 
    
#     # TODO: Create mask.
#     mask = np.arange(seq_length) < np.array(lens)[:,None]

#     # TODO: Pad
#     batch_word = arr_zeros + pad_token_id
#     batch_word[mask] = np.concatenate([i if len(i) < seq_length else i[0:seq_length]  for i in batch_word_ids] ) 
#     mask =  1*mask

#     return batch_word, mask

# class MyDataset():
#     def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
#         # Escreva aqui seu código.
#         self.examples = []
#         for text in texts:
#           token_ids = tokenize(text=text, tokenizer=tokenizer)
          
#           for j in range(len(token_ids)//(max_seq_length-1)+1):
#             input_ids = tokenize(text='[CLS]', tokenizer=tokenizer) + token_ids[j*(max_seq_length-1):(j+1)*(max_seq_length-1)]
#             target_id = token_ids[j*(max_seq_length-1):(j+1)*(max_seq_length-1)+1]

#             if (len(input_ids) < max_seq_length):
#               input_ids = input_ids + [tokenizer.pad_token_id]*(max_seq_length-len(input_ids))

#             if (len(target_id) < max_seq_length):
#               target_id = target_id + [tokenizer.pad_token_id]*(max_seq_length-len(target_id))
            
#             self.examples.append((input_ids, target_id))
           

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, idx):
#         input_ids, target_id = self.examples[idx]
#         return torch.tensor(input_ids), torch.tensor(target_id)

In [ ]:
from typing import List
from tqdm.notebook import tqdm


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int, iterador: bool):
        # Escreva aqui seu código.

        if iterador:
            self.ids = []
            for i in tqdm(range(len(texts))):
                self.ids.append(tokenizer.batch_encode_plus([texts[i]], return_tensors=None, add_special_tokens=False).input_ids[0])                
            # for i in range(len(self.ids)):
            #     self.ids[i] = self.ids[i][0]
        else:
            self.ids = tokenizer.batch_encode_plus(texts, return_tensors=None, add_special_tokens=False).input_ids

        self.iterador       = iterador
        self.dict_ids       = {}
        self.tokenizer      = tokenizer
        self.max_seq_length = max_seq_length

        total = 0

        for i in tqdm(range(len(self.ids))):
            itera = len(self.ids[i])//(self.max_seq_length-1) if len(self.ids[i])%(self.max_seq_length-1) == 0 else (len(self.ids[i])//(self.max_seq_length-1))+1
            # print(itera)
            for j in range(itera):
                self.dict_ids[total + j] = [i,j]

            total += itera

        
    def text_to_token(self, idx):

        # token_ids = tokenize(text=self.texts[idx], tokenizer=self.tokenizer)

        max0 = self.max_seq_length
        max1 = self.max_seq_length-1

        len_a = len( self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max1])
        len_b = len( self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max0])
        # print(len_a, len_b)

        limite_a = self.max_seq_length - len_a - 1
        limite_b = self.max_seq_length - len_b
        # print(limite_a, limite_b)

        p_a = self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : (self.dict_ids[idx][1]+1)*max1]
        p_b = self.ids[ self.dict_ids[idx][0] ] [self.dict_ids[idx][1]*max1 : self.dict_ids[idx][1]*max1 + max0]

        a = [self.tokenizer.cls_token_id] + p_a + [self.tokenizer.pad_token_id]*max(0, limite_a)
        b = p_b + [self.tokenizer.pad_token_id]*max(0, limite_b)

        # print(self.dict_ids[idx])
        # print(token_ids)
        # print(a)
        # print(b)

        return (torch.tensor(a), torch.tensor(b))
                
    def __len__(self):      
        return len(self.dict_ids)        

    def __getitem__(self, idx):
        return self.text_to_token(idx)

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9,iterador=True)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-06 21:25:02--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.213.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   163MB/s    in 7.3s    

2022-06-06 21:25:09 (161 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [ ]:
# # Load datasets
# max_seq_length = 9

# train_examples = 500
# valid_examples = 100
# test_examples = 100

# texts = open('sample-1gb.txt').readlines()

# print(f'Read {len(texts)} lines.')

# max_lines = train_examples + valid_examples + test_examples
# print(f'Truncating to {max_lines} lines.')
# texts = texts[:max_lines] 


# training_texts = texts[:-(valid_examples + test_examples)]
# valid_texts = texts[-(valid_examples + test_examples):-test_examples]
# test_texts = texts[-test_examples:]

# training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
# valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
# test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

In [ ]:
# Load datasets
max_seq_length = 9

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

train_examples = int(len(texts)*0.70*0.5)
valid_examples = int(len(texts)*0.15*0.5)
test_examples  = int(len(texts)*0.15*0.5)

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
print(f'Train Examples: {train_examples}')
print(f'Valid Examples: {valid_examples}')
print(f'Test Examples:  {test_examples}')
texts = texts[:max_lines]  

Read 250000 lines.
Truncating to 125000 lines.
Train Examples: 87500
Valid Examples: 18750
Test Examples:  18750


In [ ]:
# print(f'training examples: {len(training_dataset)}')
# print(f'valid examples: {len(valid_dataset)}')
# print(f'test examples: {len(test_dataset)}')

In [ ]:
print('\nLoadind Validation Texts ...')
valid_dataset    = MyDataset(texts=texts[-(valid_examples + test_examples):-test_examples], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)

print('\nLoadind Testing Texts ...')
test_dataset     = MyDataset(texts=texts[-test_examples:], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)

print('\nLoadind Training Texts ...')
training_dataset = MyDataset(texts=texts[:-(valid_examples + test_examples)], tokenizer=tokenizer, max_seq_length=max_seq_length, iterador=True)


Loadind Validation Texts ...


  0%|          | 0/18750 [00:00<?, ?it/s]

  0%|          | 0/18750 [00:00<?, ?it/s]


Loadind Testing Texts ...


  0%|          | 0/18750 [00:00<?, ?it/s]

  0%|          | 0/18750 [00:00<?, ?it/s]


Loadind Training Texts ...


  0%|          | 0/87500 [00:00<?, ?it/s]

  0%|          | 0/87500 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 12246016
valid examples: 2701364
test examples: 2568057


In [1]:
import gensim

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.
        super().__init__()

        self.max_seq_length = max_seq_length
        self.embeddings_dim = dim
        self.embeddings = nn.Embedding(vocab_size, dim)
        self.n_layers = n_layers
        self.vocab_size = vocab_size
        self.pad_token_id = pad_token_id

        self.W_q = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False) 
        self.W_k = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False) 
        self.W_v = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False) 
        self.W_o = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False)

        hidden_size = 2*self.embeddings_dim

        self.layer1 = nn.Linear(self.embeddings_dim, hidden_size)
        self.layer2 = nn.Linear(hidden_size,self.embeddings_dim)

        self.linear_output = nn.Linear(self.embeddings_dim, self.vocab_size)

    def Attention(self, q, k, v, inputs):

        scores = torch.matmul(q, k.transpose(2,1))# shape = B,L,L

        mask_pad = inputs != self.pad_token_id
        scores = scores.masked_fill(~mask_pad[:, None], -1e9)

        mask_causal = torch.tril(torch.ones(self.max_seq_length, self.max_seq_length)).to(device)
        mask_causal = mask_causal.bool()

        scores = scores.masked_fill(~mask_causal, -1e9)

        probs =  F.softmax(scores) # B,L,L
        E  = torch.matmul(probs, v) # shape = B,L,E

        return E

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        # Escreva seu código aqui.

        X_emb = self.embeddings(inputs)

        for i in range(0, self.n_layers):
          residual = X_emb

          q = self.W_q(X_emb).to(device) #Shape = (B,L,D)
          k = self.W_k(X_emb).to(device) #Shape = (B,L,D)
          v = self.W_v(X_emb).to(device) #Shape = (B,L,D)

          X_emb = self.Attention(q=q, k=k, v=v, inputs = inputs)

          X_emb = self.W_o(X_emb).to(device)

          X_emb += residual

        hidden = torch.relu(self.layer1(X_emb))
        
        out = self.layer2(hidden)


        # out_ff = torch.nn.functional.layer_norm(out + X_emb, out.shape[-1:])
        
        logits = self.linear_output(out) # logits shape: (B, L, V)

        return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3876386


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    print(logits.shape)
    print(target.shape)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

torch.Size([9000, 29794])
torch.Size([9000])
my perplexity:              35116
correct initial perplexity: 29794
Passou o no assert da perplexidade


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Laço de Treinamento e Validação

In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=tokenizer.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=tokenizer.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 steps; 0 examples so far; train ppl: 34351.79, valid ppl: 34464.44
10000 steps; 10240000 examples so far; train ppl: 425.10, valid ppl: 178.55
20000 steps; 20480000 examples so far; train ppl: 112.80, valid ppl: 85.84
30000 steps; 30720000 examples so far; train ppl: 59.04, valid ppl: 46.47
40000 steps; 40960000 examples so far; train ppl: 36.34, valid ppl: 31.48
50000 steps; 51200000 examples so far; train ppl: 26.21, valid ppl: 23.91
60000 steps; 61440000 examples so far; train ppl: 19.58, valid ppl: 18.04
70000 steps; 71680000 examples so far; train ppl: 15.20, valid ppl: 14.46
80000 steps; 81920000 examples so far; train ppl: 12.46, valid ppl: 11.89
90000 steps; 92160000 examples so far; train ppl: 10.44, valid ppl: 10.18
100000 steps; 102400000 examples so far; train ppl: 9.35, valid ppl: 9.74
110000 steps; 112640000 examples so far; train ppl: 8.47, valid ppl: 8.31
120000 steps; 122880000 examples so far; train ppl: 7.49, valid ppl: 7.54
130000 steps; 133120000 examples so far;

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test perplexity: 4047782.955909039


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me fazólio
Eu gosto de comer pizza pois me fazólio Curso
Eu gosto de comer pizza pois me fazólio Curso atentado
Eu gosto de comer pizza pois me fazólio Curso atentado Fundação
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador esteja
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador esteja me
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador esteja me faz
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador esteja me fazí
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvador esteja me fazí Curso
Eu gosto de comer pizza pois me fazólio Curso atentado Fundaçãoite somente Salvado

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
prompt = 'Sem representante brasileiro, foi realizada nesta quarta-feira'
max_output_tokens = 20

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Sem representante brasileiro, foi realizada nesta quarta - feira vasto
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK]
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK]
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK]
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK]
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK] [UNK]
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK] [UNK].
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK] [UNK]..
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK] [UNK].. Estava
Sem representante brasileiro, foi realizada nesta quarta - feira vasto [UNK] [UNK] [UNK] [UNK] [UNK].. Estava [UNK]
Sem representante brasileiro, foi realizada nesta qua

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.